<a href="https://colab.research.google.com/github/AyishaR/dicom-parse/blob/main/dicom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydicom

In [ ]:
import pydicom
from pydicom.data import get_testdata_file
import matplotlib.pyplot as plt
import pandas as pd
import csv
import os

In [ ]:
# CSV file containing all DICOM tags
dicom_tags = pd.read_csv('dicom_tags.data', sep="|")
dicom_tags

In [ ]:
# Index of tags to record
tags_to_record = list(dicom_tags.Tag)
 
# Remove the index of tag - PixelData
# The content of this tag will be stored as images inna separate folder
tags_to_record.remove('(7FE0,0010)')
 
# Creating a reverse dictionary where the key is the index of the tag and the value is the index of the element in the array/list.
dicom_tags_index_reverse = {x:i for i,x in enumerate(tags_to_record)}
dicom_tags_index_reverse

In [ ]:
 
# Path to folder with DICOM files
path = '/content/drive/My Drive/dcm_files/'
 
# Path to folder to store generated files
pathx = '/content/drive/My Drive/dcm_files/'
 
csv_filename =  'dicom_test.csv'
 
# New CSV file
with open(path + csv_filename, 'w') as csvfile:
 
    # Create CSV writer
    csvwriter = csv.writer(csvfile)
 
    # Elements in the header of the CSV
    header_row = ['Filename']    # DICOM filename
    header_row.extend(list(dicom_tags.Keyword))    # Keywords of tags
    header_row.extend(['Private'])    # Add element to keep track of private elements
    header_row.remove('PixelData')    # Remove PixelData tag
 
    csvwriter.writerow(header_row)
 
    print('Writing to file ', csv_filename)
 
    # for each file
    for filename in os.listdir(path):
        
        print('Currently reading', filename)
 
        newline = [None] * (len(header_row))    # One for each attribute to be stored
        
        newline[0] = filename    # Set filename attribute
 
        dcm_file = pydicom.dcmread(path + filename)
 
        # The features defined in the file
        features_present = list(dcm_file.keys())
 
        # Path to store image files
        folder_path = path + filename[:-4]
        os.makedirs(folder_path)
 
        private_tags = []    # To keep track of private tags
 
        for x in features_present:
 
            # Edit to match the keys of the dicom_tags_index_reverse dict
            xtag = str(x).upper().replace(" ", "")
            
            # Retrieve value
            val = dcm_file[x].value
 
            # If PixelData
            if xtag == '(7FE0,0010)':
 
                p = dcm_file.pixel_array
 
                print('Parsing image(s)')
 
                if len(p.shape)>2:
                    print("Multiple image files present")
                    for i in range(p.shape[0]):
                        plt.axis(False)
                        plt.imshow(p[i], cmap=plt.cm.gray)
                        file  = folder_path + '/' + 'Image' + str(i) + '.jpg'
                        print('Saving', file)
                        plt.savefig(file, dpi=500, bbox_inches='tight',pad_inches=0)
                else:
                    print("Single image file present")
                    plt.axis(False)
                    plt.imshow(p, plt.cm.gray)
                    file  = folder_path + '/' + 'Image.jpg'
                    print('Saving', file)
                    plt.savefig(file, dpi=500, bbox_inches='tight',pad_inches=0)
 
            else:
                # If private tag
                if xtag not in dicom_tags_index_reverse:
                    # Add to list of private tags
                    private_tags.append(dcm_file[x])
                else:
                    # Store value at location in newline array
                    newline[dicom_tags_index_reverse[xtag]+1] = val
        
        # Store the private tags at location in newline array
        newline[-1] = str(private_tags)
        
        # Write to csv file
        csvwriter.writerow(newline)

In [ ]:

# View generated CSV file 
pd.read_csv(path + 'dicom_test.csv')